In [ ]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch
import textwrap

In [ ]:
# Load pre-trained model
model_name = "deepset/bert-base-cased-squad2"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

In [ ]:
def get_answer(question, context):
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, max_length=512)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    start_idx = torch.argmax(outputs.start_logits)
    end_idx = torch.argmax(outputs.end_logits) + 1
    
    # Calculate confidence scores
    start_prob = torch.nn.functional.softmax(outputs.start_logits, dim=1)[0][start_idx].item()
    end_prob = torch.nn.functional.softmax(outputs.end_logits, dim=1)[0][end_idx-1].item()
    confidence = (start_prob + end_prob) / 2
    
    answer_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx])
    return tokenizer.convert_tokens_to_string(answer_tokens).strip(), confidence

In [ ]:
def pretty_print(text):
    print(textwrap.fill(text, width=100))

In [ ]:
# Predefined context
context = (
    "Python is a high-level, interpreted programming language created by Guido van Rossum. "
    "It supports multiple programming paradigms, including structured, object-oriented, and functional programming. "
    "Python is widely used in web development, AI, scientific computing, data analysis, and more."
)

questions = [
    "Who created Python?",
    "What is Python used for?",
    "What kind of programming paradigms does Python support?"
]

In [ ]:
print("=== Question Answering using BERT ===\n")
pretty_print("Context:\n" + context + "\n")

for q in questions:
    answer, score = get_answer(q, context)
    print(f"\nQuestion: {q}")
    print(f"Answer: {answer}")
    print(f"Confidence: {score:.2f}")

In [ ]:
# Optional user input
print("\n=== Try your own question! ===")
user_question = input("Enter your question: ")
answer, score = get_answer(user_question, context)
print(f"\nAnswer: {answer}")
print(f"Confidence: {score:.2f}")